In [1]:
%run ./../../utils/commonImports.py
%run ./../../utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.5, period=None):
    data = load_trading_data(db, table, from_date, to_date, period)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2014-01-01', period='5min')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()

In [4]:
data.head()

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date
1,1424372400,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:00:00
2,1424373000,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:10:00
3,1424373300,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:15:00
4,1424373600,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:20:00
5,1424373900,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:25:00


# Experiment

Celý experiment spočívá v olabelovaní dat následujícím způsobem. Pokud daný bod při nákupu vydělá více než 1% tak True.
Snaha bude naučit klasifikátor tak, aby měl co nejlepší precision a zároveň provedl co nejvíce obchodů.

# Calculation

In [13]:
import time

maxs = []
till_maxs = []
mins = []
df = ohlc.copy()
DAY = 288
TARGET = 0.01

def _max_row(day_df, col):
    i = day_df[col].idxmax()
    max_row = day_df.iloc[i, :]
    return max_row

def _min_row(day_df, col):
    i = day_df[col].idxmin()
    min_row = day_df.iloc[i, :]
    return min_row

def _max(day_df):
    close_max_row = _max_row(day_df, 'close')
    open_max_row = _max_row(day_df, 'open')
    return (close_max_row, 'close') if close_max_row['close'] > open_max_row['open'] else (open_max_row, 'open')

def _fullfiled_row(row, day_df, col, target):
    day_df['R'] = _return(row['close'], day_df[col])
    fullfiled = day_df.query(f"R >= {target}")
    if fullfiled.shape[0] > 0:
        return fullfiled.iloc[0, :]
    else:
        return day_df.iloc[-1, :]

def fullfiled(row, day_df, target):
    close_row = _fullfiled_row(row, day_df, 'close', target)
    open_row = _fullfiled_row(row, day_df, 'open', target)
    return (close_row, 'close') if close_row['close'] > open_row['open'] else (open_row, 'open')

def _return(v1, v2):
    return (v2 / v1) - 1


start_time = time.time()
for i in range(df.shape[0] - 1):
    if i % 1000 == 0:
        print(i, '%.3f s' % (time.time() - start_time))
        start_time = time.time()
    
    day_i = i + DAY if i + DAY < df.shape[0] else df.shape[0]-1
    row = df.iloc[i, :]
    day_df = df.iloc[i+1:day_i, :].reset_index().drop('index', axis=1).reset_index()

    
    fullfiled_row, fullfiled_col = fullfiled(row, day_df, TARGET)
    
    max_index = fullfiled_row['index'] if fullfiled_row['index'] > 0 else 1
    day_df = day_df.iloc[:max_index, :]
    min_row = _min_row(day_df, 'low')
    
    maxs.append(_return(row['close'], fullfiled_row[fullfiled_col]))
    till_maxs.append(fullfiled_row['index'])
    mins.append(_return(row['close'], min_row['low']))

0 0.001 s
1000 10.664 s
2000 9.985 s
3000 9.910 s
4000 10.110 s
5000 10.306 s
6000 9.965 s
7000 10.074 s
8000 10.945 s
9000 10.216 s
10000 9.949 s
11000 11.019 s
12000 10.188 s
13000 9.968 s
14000 9.921 s
15000 10.669 s
16000 10.459 s
17000 12.183 s
18000 11.687 s
19000 12.552 s
20000 10.233 s
21000 9.944 s
22000 10.271 s
23000 10.266 s
24000 9.841 s
25000 9.816 s
26000 9.729 s
27000 9.782 s
28000 10.233 s
29000 10.986 s
30000 10.947 s
31000 9.925 s
32000 9.831 s
33000 9.990 s
34000 9.881 s
35000 9.973 s
36000 9.869 s
37000 9.871 s
38000 10.605 s
39000 9.860 s
40000 9.885 s
41000 9.911 s
42000 9.814 s
43000 9.826 s
44000 9.828 s
45000 9.839 s
46000 9.865 s
47000 9.816 s
48000 9.881 s
49000 9.851 s
50000 9.877 s
51000 9.859 s
52000 9.949 s
53000 9.990 s
54000 9.838 s
55000 9.820 s
56000 9.854 s
57000 9.834 s
58000 9.855 s
59000 9.858 s
60000 9.828 s
61000 9.805 s
62000 9.775 s
63000 9.865 s
64000 10.408 s
65000 9.835 s
66000 9.796 s
67000 9.922 s
68000 11.287 s
69000 10.047 s
70000 11.4

IndexError: single positional indexer is out-of-bounds

In [14]:
maxs += [-1, -1]
till_maxs += [-1, -1]
mins += [-1, -1]

In [15]:
df = ohlc.copy()
df['maxReturn'] = maxs
df['tillMax'] = till_maxs
df['minReturn'] = mins

In [16]:
df

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,maxReturn,tillMax,minReturn
1,1424372400,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:00:00,0.0667,15,0.0000
2,1424373000,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:10:00,0.0667,14,0.0000
3,1424373300,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:15:00,0.0667,13,0.0000
4,1424373600,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:20:00,0.0667,12,0.0000
5,1424373900,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:25:00,0.0667,11,0.0000
7,1424374200,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:30:00,0.0667,10,0.0000
8,1424374500,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:35:00,0.0667,9,0.0000
9,1424374800,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:40:00,0.0667,8,0.0000
10,1424375100,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:45:00,0.0667,7,0.0000
11,1424375400,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:50:00,0.0667,6,0.0000


In [17]:
write_tsv(df, 'D:\\Dropbox\\My work\\krypl-project\\analysis\\return-01-within-day.tsv')